In [70]:
import pandas as pd
import requests
import matplotlib
import pymongo
import numpy as np
import re
import pickle as pkl
import json

In [71]:
MONGO_CONNECT_URL="mongodb://localhost:27017/IR_Project"
db_name = "ir_535"
client = pymongo.MongoClient(MONGO_CONNECT_URL)
db = client.get_database(db_name)

POI LIST

In [43]:
poi_tweets = list(db['tweets'].find({
    "poi_name": {"$exists": True}
}))
poi_names_all = [tweet['poi_name'] for tweet in poi_tweets]
poi_names = list(np.unique(poi_names_all))
poi_names

['ArvindKejriwal',
 'BernieSanders',
 'CDCgov',
 'Claudiashein',
 'EnriqueAlfaroR',
 'GovRonDeSantis',
 'GregAbbott_TX',
 'HHS_ASH',
 'HLGatell',
 'JoeBiden',
 'KamalaHarris',
 'LindseyGrahamSC',
 'MamataOfficial',
 'MoHFW_INDIA',
 'POTUS',
 'PeteButtigieg',
 'RahulGandhi',
 'RandPaul',
 'RicardoMonrealA',
 'SSalud_mx',
 'SecBecerra',
 'SenSanders',
 'VP',
 'VicenteFoxQue',
 'alfredodelmazo',
 'alitomorenoc',
 'amyklobuchar',
 'fernandeznorona',
 'lopezobrador_',
 'm_ebrard',
 'mansukhmandviya',
 'marcorubio',
 'mattgaetz',
 'mtgreenee',
 'narendramodi',
 'sambitswaraj',
 'tatclouthier',
 'tedcruz']

Find Sentiment of tweets across the entire corpus

In [72]:

data_entire_corpus = {
    "POSITIVE" :  32574,
    "NEGATIVE" : 54124,
    "NEUTRAL" :  135416,
    "MIXED" : 8996
}

In [73]:
# FOR COUNTRY based individual sentiments.
country_based_sentiments = {}
for country in ["USA", "India", "Mexico"]:
    country_based_sentiments[country] = {}
    for sentiment in ["POSITIVE", "NEGATIVE", "NEUTRAL", "MIXED"]:
        country_based_sentiments[country][sentiment] = {}
        out = list(db['tweets'].aggregate([
            {
                '$match': {
                    'country': re.compile(r""+country+r"(?i)"), 
                    'sentiment': sentiment
                }
            }, {
                '$count': 'sentiment'
            }
        ]))
        country_based_sentiments[country][sentiment] = out[0]['sentiment']

In [74]:
country_based_sentiments

{'USA': {'POSITIVE': 17468,
  'NEGATIVE': 40110,
  'NEUTRAL': 78148,
  'MIXED': 2966},
 'India': {'POSITIVE': 8052, 'NEGATIVE': 4899, 'NEUTRAL': 28782, 'MIXED': 342},
 'Mexico': {'POSITIVE': 6957,
  'NEGATIVE': 8886,
  'NEUTRAL': 28124,
  'MIXED': 5668}}

In [75]:
# Calculated DATA 
country_based_sentiments = {
    'USA': {
        'POSITIVE': 17468,
        'NEGATIVE': 40110,
        'NEUTRAL': 78148,
        'MIXED': 2966
    },
    'India': {
        'POSITIVE': 8052,
        'NEGATIVE': 4899,
        'NEUTRAL': 28782,
        'MIXED': 342
    },
    'Mexico': {
        'POSITIVE': 6957,
        'NEGATIVE': 8886,
        'NEUTRAL': 28124,
        'MIXED': 5668
    }
}

# POI DATA

In [76]:
poi_tweet_based_sentiments = {}
for poi in poi_names:
    poi_tweet_based_sentiments[poi] = {}
    for sentiment in ["POSITIVE", "NEGATIVE", "NEUTRAL", "MIXED"]:
        poi_tweet_based_sentiments[poi][sentiment] = {}
        out = list(db['tweets'].aggregate([
            {
                '$match': {
                    'poi_name': poi, 
                    'sentiment': sentiment
                }
            }, {
                '$count': 'sentiment'
            }
        ]))
        if len(out) != 0:
            poi_tweet_based_sentiments[poi][sentiment] = out[0]['sentiment']
        else:
            print(f"poi_name: {poi} || sentiment : {sentiment}")
            poi_tweet_based_sentiments[poi][sentiment] = 0

poi_name: MoHFW_INDIA || sentiment : MIXED


In [77]:
poi_tweet_based_sentiments = {
    'ArvindKejriwal': {
        'POSITIVE': 549,
        'NEGATIVE': 168,
        'NEUTRAL': 708,
        'MIXED': 15
    },
    'BernieSanders': {
        'POSITIVE': 164,
        'NEGATIVE': 490,
        'NEUTRAL': 663,
        'MIXED': 79
    },
    'CDCgov': {
        'POSITIVE': 70,
        'NEGATIVE': 58,
        'NEUTRAL': 1693,
        'MIXED': 14
    },
    'Claudiashein': {
        'POSITIVE': 242,
        'NEGATIVE': 6,
        'NEUTRAL': 439,
        'MIXED': 32
    },
    'EnriqueAlfaroR': {
        'POSITIVE': 162,
        'NEGATIVE': 36,
        'NEUTRAL': 406,
        'MIXED': 53
    },
    'GovRonDeSantis': {
        'POSITIVE': 398,
        'NEGATIVE': 66,
        'NEUTRAL': 663,
        'MIXED': 8
    },
    'GregAbbott_TX': {
        'POSITIVE': 348,
        'NEGATIVE': 255,
        'NEUTRAL': 621,
        'MIXED': 10
    },
    'HHS_ASH': {
        'POSITIVE': 181,
        'NEGATIVE': 34,
        'NEUTRAL': 559,
        'MIXED': 6
    },
    'HLGatell': {
        'POSITIVE': 48,
        'NEGATIVE': 15,
        'NEUTRAL': 724,
        'MIXED': 6
    },
    'JoeBiden': {
        'POSITIVE': 334,
        'NEGATIVE': 251,
        'NEUTRAL': 925,
        'MIXED': 39
    },
    'KamalaHarris': {
        'POSITIVE': 399,
        'NEGATIVE': 238,
        'NEUTRAL': 889,
        'MIXED': 17
    },
    'LindseyGrahamSC': {
        'POSITIVE': 193,
        'NEGATIVE': 406,
        'NEUTRAL': 528,
        'MIXED': 47
    },
    'MamataOfficial': {
        'POSITIVE': 376,
        'NEGATIVE': 98,
        'NEUTRAL': 242,
        'MIXED': 17
    },
    'MoHFW_INDIA': {
        'POSITIVE': 185,
        'NEGATIVE': 50,
        'NEUTRAL': 1687,
        'MIXED': 0
    },
    'POTUS': {
        'POSITIVE': 262,
        'NEGATIVE': 92,
        'NEUTRAL': 566,
        'MIXED': 40
    },
    'PeteButtigieg': {
        'POSITIVE': 317,
        'NEGATIVE': 205,
        'NEUTRAL': 452,
        'MIXED': 15
    },
    'RahulGandhi': {
        'POSITIVE': 327,
        'NEGATIVE': 556,
        'NEUTRAL': 520,
        'MIXED': 33
    },
    'RandPaul': {
        'POSITIVE': 207,
        'NEGATIVE': 344,
        'NEUTRAL': 556,
        'MIXED': 19
    },
    'RicardoMonrealA': {
        'POSITIVE': 145,
        'NEGATIVE': 36,
        'NEUTRAL': 759,
        'MIXED': 55
    },
    'SSalud_mx': {
        'POSITIVE': 570,
        'NEGATIVE': 11,
        'NEUTRAL': 1670,
        'MIXED': 241
    },
    'SecBecerra': {
        'POSITIVE': 175,
        'NEGATIVE': 25,
        'NEUTRAL': 232,
        'MIXED': 10
    },
    'SenSanders': {
        'POSITIVE': 90,
        'NEGATIVE': 411,
        'NEUTRAL': 348,
        'MIXED': 60
    },
    'VP': {
        'POSITIVE': 272,
        'NEGATIVE': 47,
        'NEUTRAL': 411,
        'MIXED': 16
    },
    'VicenteFoxQue': {
        'POSITIVE': 243,
        'NEGATIVE': 529,
        'NEUTRAL': 339,
        'MIXED': 236
    },
    'alfredodelmazo': {
        'POSITIVE': 244,
        'NEGATIVE': 4,
        'NEUTRAL': 1407,
        'MIXED': 8
    },
    'alitomorenoc': {
        'POSITIVE': 761,
        'NEGATIVE': 93,
        'NEUTRAL': 296,
        'MIXED': 67
    },
    'amyklobuchar': {
        'POSITIVE': 509,
        'NEGATIVE': 241,
        'NEUTRAL': 660,
        'MIXED': 52
    },
    'fernandeznorona': {
        'POSITIVE': 315,
        'NEGATIVE': 383,
        'NEUTRAL': 781,
        'MIXED': 207
    },
    'lopezobrador_': {
        'POSITIVE': 274,
        'NEGATIVE': 75,
        'NEUTRAL': 1126,
        'MIXED': 105
    },
    'm_ebrard': {
        'POSITIVE': 450,
        'NEGATIVE': 76,
        'NEUTRAL': 999,
        'MIXED': 88
    },
    'mansukhmandviya': {
        'POSITIVE': 1420,
        'NEGATIVE': 87,
        'NEUTRAL': 1640,
        'MIXED': 3
    },
    'marcorubio': {
        'POSITIVE': 122,
        'NEGATIVE': 350,
        'NEUTRAL': 552,
        'MIXED': 28
    },
    'mattgaetz': {
        'POSITIVE': 210,
        'NEGATIVE': 367,
        'NEUTRAL': 584,
        'MIXED': 8
    },
    'mtgreenee': {
        'POSITIVE': 214,
        'NEGATIVE': 886,
        'NEUTRAL': 387,
        'MIXED': 33
    },
    'narendramodi': {
        'POSITIVE': 872,
        'NEGATIVE': 55,
        'NEUTRAL': 929,
        'MIXED': 19
    },
    'sambitswaraj': {
        'POSITIVE': 394,
        'NEGATIVE': 82,
        'NEUTRAL': 1171,
        'MIXED': 5
    },
    'tatclouthier': {
        'POSITIVE': 166,
        'NEGATIVE': 33,
        'NEUTRAL': 382,
        'MIXED': 22
    },
    'tedcruz': {
        'POSITIVE': 163,
        'NEGATIVE': 591,
        'NEUTRAL': 614,
        'MIXED': 20
    }
}

In [78]:
poi_country = {
    'CDCgov': 'USA',
    'JoeBiden': 'USA',
    'KamalaHarris': 'USA',
    'LindseyGrahamSC': 'USA',
    'BernieSanders': 'USA',
    'tedcruz': 'USA',
    'GovRonDeSantis': 'USA',
    'GregAbbott_TX': 'USA',
    'HHS_ASH': 'USA',
    'marcorubio': 'USA',
    'mattgaetz': 'USA',
    'mtgreenee': 'USA',
    'RandPaul': 'USA',
    'SecBecerra': 'USA',
    'SenSanders': 'USA',
    'VP': 'USA',
    'amyklobuchar': 'USA',
    'ArvindKejriwal': 'INDIA',
    'MamataOfficial': 'INDIA',
    'MoHFW_INDIA': 'INDIA',
    'RahulGandhi': 'INDIA',
    'mansukhmandviya': 'INDIA',
    'narendramodi': 'INDIA',
    'sambitswaraj': 'INDIA',
    'RicardoMonrealA': 'MEXICO',
    'SSalud_mx': 'MEXICO',
    'VicenteFoxQue': 'MEXICO',
    'alfredodelmazo': 'MEXICO',
    'alitomorenoc': 'MEXICO',
    'fernandeznorona': 'MEXICO',
    'lopezobrador_': 'MEXICO',
    'm_ebrard': 'MEXICO',
    'tatclouthier': 'MEXICO',
    'Claudiashein': 'MEXICO',
    'EnriqueAlfaroR': 'MEXICO',
    'HLGatell': 'MEXICO',
}

In [79]:
countryWise = {
    "USA": [],
    "INDIA": [],
    "MEXICO": [],
}
for key, value in poi_country.items():
    countryWise[value].append(key)

In [80]:
countryWise

{'USA': ['CDCgov',
  'JoeBiden',
  'KamalaHarris',
  'LindseyGrahamSC',
  'BernieSanders',
  'tedcruz',
  'GovRonDeSantis',
  'GregAbbott_TX',
  'HHS_ASH',
  'marcorubio',
  'mattgaetz',
  'mtgreenee',
  'RandPaul',
  'SecBecerra',
  'SenSanders',
  'VP',
  'amyklobuchar'],
 'INDIA': ['ArvindKejriwal',
  'MamataOfficial',
  'MoHFW_INDIA',
  'RahulGandhi',
  'mansukhmandviya',
  'narendramodi',
  'sambitswaraj'],
 'MEXICO': ['RicardoMonrealA',
  'SSalud_mx',
  'VicenteFoxQue',
  'alfredodelmazo',
  'alitomorenoc',
  'fernandeznorona',
  'lopezobrador_',
  'm_ebrard',
  'tatclouthier',
  'Claudiashein',
  'EnriqueAlfaroR',
  'HLGatell']}

# HASHTAGS

In [81]:
response = list(db['tweets'].aggregate([
    {
        '$match': {
            'hashtags': {
                '$exists': True
            }
        }
    }
]))
hashtags_list_all = [item["hashtags"] for item in response]
hashtags_flat = [item for sublist in hashtags_list_all for item in sublist]
hashtags_li , hashtags_count = np.unique(hashtags_flat, return_counts=True)
hashtags_li = hashtags_li.tolist()
hashtags_count = hashtags_count.tolist()
hashtags_word_colud_corpus = []
for i in range(0, len(hashtags_count)):
    hashtags_word_colud_corpus.append({
        "name": hashtags_li[i],
        "weight": hashtags_count[i],
    })
sorted_hashtags_all = sorted(hashtags_word_colud_corpus, key= lambda k: k['weight'], reverse=True)
len(sorted_hashtags_all)


43001

# COUNTRY WISE HASHTAGS

In [82]:
country_wise_hashtags = {}
for country in ["USA", "India", "Mexico"]:
    response = list(db['tweets'].aggregate([
        {
            '$match': {
                "country": re.compile(r""+country+r"(?i)"),
                'hashtags': {
                    '$exists': True
                }
            }
        }
    ]))
    hashtags_list_all = [item["hashtags"] for item in response]
    hashtags_flat = [item for sublist in hashtags_list_all for item in sublist]
    hashtags_li , hashtags_count = np.unique(hashtags_flat, return_counts=True)
    hashtags_li = hashtags_li.tolist()
    hashtags_count = hashtags_count.tolist()
    hashtags_word_colud_corpus = []
    for i in range(0, len(hashtags_count)):
        hashtags_word_colud_corpus.append({
            "name": hashtags_li[i],
            "weight": hashtags_count[i],
        })
    sorted_hashtags_country = sorted(hashtags_word_colud_corpus, key= lambda k: k['weight'], reverse=True)
    len(sorted_hashtags_country)
    country_wise_hashtags[country] = sorted_hashtags_country


In [83]:
country_wise_hashtags

{'USA': [{'name': 'COVID19', 'weight': 7459},
  {'name': 'GetVaccinated', 'weight': 2075},
  {'name': 'DeltaVariant', 'weight': 1805},
  {'name': 'IndiaFightsCorona', 'weight': 1281},
  {'name': 'WearAMask', 'weight': 1222},
  {'name': 'VaccineMandate', 'weight': 1170},
  {'name': 'VaccinePassport', 'weight': 1170},
  {'name': 'VaccinesWork', 'weight': 1160},
  {'name': 'CovidVaccine', 'weight': 1065},
  {'name': 'VaccinationDrive', 'weight': 991},
  {'name': 'LargestVaccineDrive', 'weight': 985},
  {'name': 'vaccine', 'weight': 920},
  {'name': 'LargestVaccinationDrive', 'weight': 830},
  {'name': 'COVID', 'weight': 823},
  {'name': 'COVISHIELD', 'weight': 808},
  {'name': 'Vaccine', 'weight': 803},
  {'name': 'NoVaccinePassports', 'weight': 786},
  {'name': 'COVAXIN', 'weight': 731},
  {'name': 'CoWIN', 'weight': 731},
  {'name': 'BBMP', 'weight': 705},
  {'name': 'coronavirus', 'weight': 702},
  {'name': 'VaccinesSaveLives', 'weight': 694},
  {'name': 'Covid_19', 'weight': 685},
  {

In [ ]:
[
    {
        '$match': {
            'poi_name': 'ArvindKejriwal'
        }
    }, {
        '$project': {
            'date': {
                '$dateFromString': {
                    'dateString': '$tweet_date'
                }
            }
        }
    }, {
        '$group': {
            '_id': {
                '$dateToString': {
                    'date': '$date', 
                    'format': '%Y-%m-%d'
                }
            }, 
            'count_tweets': {
                '$sum': 1
            }
        }
    }, {
        '$project': {
            'date': {
                '$dateFromString': {
                    'dateString': '$_id'
                }
            }, 
            'count_tweets': '$count_tweets'
        }
    }, {
        '$sort': {
            'date': 1
        }
    }
]


In [14]:
[
        {
            '$match': {'poi_name': poi_name}}, {
            '$project': {'date': {'$dateFromString': {'dateString': '$tweet_date'}}}
        }, {
            '$group': {
                '_id': {
                    'month': {'$month': '$date'}, 
                    'day': {'$dayOfMonth': '$date'}, 
                    'year': {'$year': '$date'}
                }, 
                'count_tweets': {'$sum': 1}
    }}]

NameError: name 'poi_name' is not defined

# POI vs no. of tweets

In [84]:
poi_tweet_count_response = {}
for poi_name in poi_names:
    poi_tweet_count_response[poi_name] = list(db['tweets'].aggregate([
    {
        '$match': {
            'poi_name': poi_name
        }
    }, {
        '$project': {
            'date': {
                '$dateFromString': {
                    'dateString': '$tweet_date'
                }
            }
        }
    }, {
        '$group': {
            '_id': {
                '$dateToString': {
                    'date': '$date', 
                    'format': '%Y-%m-%d'
                }
            }, 
            'count_tweets': {
                '$sum': 1
            }
        }
    }, {
        '$project': {
            'date': {
                '$dateFromString': {
                    'dateString': '$_id'
                }
            }, 
            'count_tweets': '$count_tweets'
        }
    }, {
        '$sort': {
            'date': 1
        }
    }
    ]))

In [85]:
poi_tweet_count_mod = {}

for poi_name in poi_names:
    res_list = poi_tweet_count_response[poi_name]
    date_list = []
    count_list = []
    for data in res_list:
        date = f"{str(data['date'])[:10]}"
        # date = f"{data['_id']['month']:02}-{data['_id']['day']:02}-{data['_id']['year']:04}"
        # print(date)
        # break
        date = date[5:7]+"-"+date[8:10]+"-"+date[:4]
        # print(date)
        # break
        
        date_list.append(date)
        count_list.append(data['count_tweets'])
    poi_tweet_count_mod[poi_name] = {
        "date_list": date_list,
        "count_list": count_list
    }

In [86]:
poi_tweet_count_mod

{'ArvindKejriwal': {'date_list': ['10-11-2020',
   '10-12-2020',
   '10-13-2020',
   '10-14-2020',
   '10-15-2020',
   '10-16-2020',
   '10-17-2020',
   '10-18-2020',
   '10-19-2020',
   '10-20-2020',
   '10-21-2020',
   '10-22-2020',
   '10-23-2020',
   '10-24-2020',
   '10-25-2020',
   '10-27-2020',
   '10-28-2020',
   '10-29-2020',
   '10-30-2020',
   '10-31-2020',
   '11-01-2020',
   '11-02-2020',
   '11-03-2020',
   '11-04-2020',
   '11-05-2020',
   '11-06-2020',
   '11-07-2020',
   '11-08-2020',
   '11-09-2020',
   '11-10-2020',
   '11-11-2020',
   '11-12-2020',
   '11-13-2020',
   '11-14-2020',
   '11-15-2020',
   '11-16-2020',
   '11-17-2020',
   '11-18-2020',
   '11-19-2020',
   '11-20-2020',
   '11-22-2020',
   '11-23-2020',
   '11-25-2020',
   '11-26-2020',
   '11-27-2020',
   '11-28-2020',
   '11-29-2020',
   '11-30-2020',
   '12-01-2020',
   '12-02-2020',
   '12-03-2020',
   '12-04-2020',
   '12-06-2020',
   '12-07-2020',
   '12-08-2020',
   '12-09-2020',
   '12-10-2020',


# POI WISE HASHTAGS

In [87]:
poi_wise_hashtags = {}
for poi in poi_names:
    response = list(db['tweets'].aggregate([
        {
            '$match': {
                "poi_name": re.compile(r""+poi+r"(?i)"),
                'hashtags': {
                    '$exists': True
                }
            }
        }
    ]))
    hashtags_list_all = [item["hashtags"] for item in response]
    hashtags_flat = [item for sublist in hashtags_list_all for item in sublist]
    hashtags_li , hashtags_count = np.unique(hashtags_flat, return_counts=True)
    hashtags_li = hashtags_li.tolist()
    hashtags_count = hashtags_count.tolist()
    hashtags_poi = []
    for i in range(0, len(hashtags_count)):
        hashtags_poi.append({
            "name": hashtags_li[i],
            "weight": hashtags_count[i],
        })
    sorted_hashtags_poi = sorted(hashtags_poi, key= lambda k: k['weight'], reverse=True)
    len(sorted_hashtags_poi)
    poi_wise_hashtags[poi] = sorted_hashtags_poi


In [91]:
# SENTIMENT TOP HASHTAGS

In [94]:
sentiment_wise_hashtags = {}
for sentiment in ["POSITIVE", "NEGATIVE", "NEUTRAL", "MIXED"]:
    response = list(db['tweets'].aggregate([
        {
            '$match': {
                "sentiment": sentiment,
                'hashtags': {
                    '$exists': True
                }
            }
        }
    ]))
    hashtags_list_all = [item["hashtags"] for item in response]
    hashtags_flat = [item for sublist in hashtags_list_all for item in sublist]
    hashtags_li , hashtags_count = np.unique(hashtags_flat, return_counts=True)
    hashtags_li = hashtags_li.tolist()
    hashtags_count = hashtags_count.tolist()
    hashtags_sentiment = []
    for i in range(0, len(hashtags_count)):
        hashtags_sentiment.append({
            "name": hashtags_li[i],
            "weight": hashtags_count[i],
        })
    sorted_hashtags_sentiment = sorted(hashtags_sentiment, key= lambda k: k['weight'], reverse=True)
    len(sorted_hashtags_sentiment)
    sentiment_wise_hashtags[sentiment] = sorted_hashtags_sentiment

In [95]:
sentiment_wise_hashtags

{'POSITIVE': [{'name': 'COVID19', 'weight': 843},
  {'name': 'PorAmorALaVida', 'weight': 418},
  {'name': 'LargestVaccineDrive', 'weight': 375},
  {'name': 'GetVaccinated', 'weight': 369},
  {'name': 'LargestVaccinationDrive', 'weight': 353},
  {'name': 'MeCuido', 'weight': 353},
  {'name': 'VaccinationDrive', 'weight': 238},
  {'name': 'IndiaFightsCorona', 'weight': 220},
  {'name': 'VaccinesWork', 'weight': 215},
  {'name': 'LavadoDeManos', 'weight': 205},
  {'name': 'UsoDeCubrebocas', 'weight': 202},
  {'name': 'VaccineSeva', 'weight': 187},
  {'name': 'HappyBdayModiji', 'weight': 179},
  {'name': 'WearAMask', 'weight': 165},
  {'name': 'VaccinesSaveLives', 'weight': 141},
  {'name': 'FullyVaccinated', 'weight': 137},
  {'name': 'CovidVaccine', 'weight': 127},
  {'name': 'DeltaVariant', 'weight': 108},
  {'name': 'StaySafe', 'weight': 104},
  {'name': 'VaccinePassport', 'weight': 101},
  {'name': 'SuSanaDistancia', 'weight': 100},
  {'name': 'HappyBirthdayModiji', 'weight': 99},
  {

In [99]:
negative_hashtags = sentiment_wise_hashtags['NEGATIVE']
positive_hashtags = [h['name'] for h in sentiment_wise_hashtags['POSITIVE']]
all_negative_hashtags = []

for nh in negative_hashtags:
   if nh['name'] not in positive_hashtags:
       all_negative_hashtags.append(nh['name']) 

In [100]:
all_negative_hashtags

['VacunaciónAMenoresYa',
 'OaxacaSinSaludPorDespidos',
 'NosFaltanVacunas',
 'TRIPSWaiver',
 'Texas',
 'FoxNews',
 'SmartNews',
 'Trump',
 'Airfinity',
 'SOSCuba',
 'MakeTexasCenterWork',
 'TransferCasestoNBC',
 'adjudicate',
 'PMCaresFund_Scam',
 'save_our_job_in_hp',
 'we_want_justice',
 'BorderCrisis',
 'Nacionales',
 'PromisesBroken',
 'एचपी_फैक्टरी',
 'राष्ट्रीय_बेरोजगार_दिवस',
 '1stShotsBefore3rdShots',
 'abpoli',
 'angelmessage',
 'blackmagic',
 'illuminatiexposed',
 'quietstorm',
 'satanicritual',
 'the10commandments',
 'themosthighisincontrol',
 'thetakedown',
 'warnings',
 'worldpeace',
 'FreeTheVaccine',
 'JNJGreed',
 'MedicareForAll',
 'PharmaGreedKills',
 'VOWNow',
 'misinformation',
 'Americans',
 'Hypocrisy',
 'जीवन',
 's',
 'AntiVaxxers',
 'CubaGate',
 'Johnson',
 'TuckerCarlson',
 'AbbottFailedTexas',
 'BidenMustGo',
 'DeathDesantis',
 'Deltavariant',
 'discrimination',
 'rejectthecovidpass',
 'AlumnosEnPeligro',
 'Iran',
 'NoVaccine',
 'RemoveRon',
 'ShareTheDoses',
 

In [96]:
data = {
    "poi_names": poi_names,
    "poi_country_mapping": poi_country,
    "countryWisePOI": countryWise,
    "across_entire_corpus": data_entire_corpus,
    "country_based_sentiments": country_based_sentiments,
    "poi_tweet_based_sentiments": poi_tweet_based_sentiments,
    "hashtags_word_cloud_corpus": sorted_hashtags_all[:100],
    "poi_tweet_count": poi_tweet_count_mod,
    "country_wise_hashtags": country_wise_hashtags,
    "poi_wise_hashtags": poi_wise_hashtags,
    "sentiment_wise_hashtags": sentiment_wise_hashtags
}
f = open("sentiment_corpus.pkl", "wb")
pkl.dump(data, f, pkl.HIGHEST_PROTOCOL)
f.close()

In [97]:
f = open("sentiment_corpus.json", "w")
json.dump(data, f)
f.close()